In [7]:
import pandas as pd
import numpy as np


In [8]:
#function to combine picks and point values
def combine(df1):
    my_file = df1.copy()
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Player']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    
    my_file.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
    return my_file

#function to calculate totals
def totals(df):
    
    max_score = 0
    games = len(df.columns)-1
    points = 16
    while games > 0:
        max_score = max_score + points
        points = points - 1
        games = games - 1
    i = 4  
    games = len(df.columns)-1
    df['Score'] = 0
    df['Game Count'] = 0
    df['Max Score'] = max_score
        
    while i < 41:
        for col in df.loc[:, ~df.columns.isin(['Player', 'Score', 'Game Count', 'Max Score'])]:
            if df[col][0] is np.nan:
                break
            
            elif df[col][0] == df[col][i][0]:
                df.loc[i, 'Game Count'] = df.loc[i, 'Game Count'] + 1
                df.loc[i, 'Score'] = df.loc[i, 'Score'] + df.loc[i, col][1]
            
            else:  
                df.loc[i, 'Max Score'] = df.loc[i, 'Max Score'] - df.loc[i, col][1]
               
        i = i + 2
    
    df['Pick Percentage'] = round(df['Game Count']/games*100, 2)
    df['Points per win'] = round(df['Score']/df['Game Count'], 2)

    df1 = df[['Player', 'Score', 'Game Count', 'Max Score', 'Pick Percentage', 'Points per win']][4:]
    df1.set_index('Player', inplace=True)
    
    return df1

#upset counter
def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    dog_spread_tot = 0
    big_dog_count = 0
    spread_list = []
    for col in df.loc[:, ~df.columns.isin(['Player', 'Score', 'Max Score'])]:
        if df[col][3] < 0 and df[col][1] == df[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            dog_spread_tot = dog_spread_tot + -(df[col][3])
            spread_list.append(abs(df[col][3]))
            #print('Road upset alert ', df[col][0], "spread ", -(df[col][3]))
            if abs(df[col][3]) > 6.5:
                big_dog_count = big_dog_count + 1
                #print("big dog woof woof")
        elif df[col][3] > 0 and df[col][2][0] == df[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            dog_spread_tot = dog_spread_tot + df[col][3]
            spread_list.append(df[col][3])
            #print('Home dog win ', df[col][2])
            if abs(df[col][3]) > 6.5:
                big_dog_count = big_dog_count + 1
                #print("big dog woof woof")
        elif df[col][0] == "TIE":
            tie_count = tie_count + 1
            #print("A friggen tie...  ", df[col][2])
        else:
            fav_count = fav_count + 1
    
    dog_spread_avg = round(dog_spread_tot/upset_count, 2)
    print("Total of ", upset_count, "upsets this week. With ", big_dog_count, "being big dogs." )
    return upset_count, spread_list #dog_spread_avg, big_dog_count fav_count, tie_count


#finds avg points per win for league
def points_per_win (df):
    total_wins = df['Game Count'].sum()
    total_points = df['Score'].sum()
    avg_pts_per_win = total_points/total_wins
    points = round(avg_pts_per_win, 2)
    return points

#sum total points each week
def sum_totals(season_totals, week_stats, week_number_rank):
    season_totals[['Score', 'Game Count']] = season_totals[['Score', 'Game Count']] + week_stats[['Score', 'Game Count']]
    season_totals[week_number_rank] = season_totals['Score'].rank(ascending=False, method='min')

    return season_totals





In [10]:
#create list of result files
results_csv = ['week1_results.csv', 'week2_results.csv', 'week3_results.csv', 'week4_results.csv',
 'week5_results.csv', 'week6_results.csv', 'week7_results.csv', 'week8_results.csv']#, 'week9_results.csv']

#create results dictionary 
results = {'week1': 1, 'week2': 2, 'week3': 3, 'week4': 4, 'week5': 5, 'week6': 6, 'week7': 7, 'week8': 8} #, 'week9': 9}

#read results files and add them to results dictionary
i = 0
for week in results: 
    results[week] = combine(pd.read_csv(results_csv[i]))
    i = i + 1
    

In [20]:
week9_raw = pd.read_csv('week9_results.csv')
week9_raw.head()
#week9 = combine(week9_raw)

,Players,PHI at HOU,LAC at ATL,GB at DET,MIA at CHI,LV at JAX,IND at NE,BUF at NYJ,CAR at CIN,MIN at WAS,SEA at ARI,LAR at TB,TEN at KC,BAL at NO
0,Winner,PHI,LAC,DET,MIA,JAX,NE,NYJ,CIN,MIN,NaN,NaN,NaN,NaN
1,Away Team,PHI,LAC,GB,MIA,LV,IND,BUF,CAR,MIN,SEA,LAR,TEN,BAL
2,Home Team,HOU,ATL,DET,CHI,JAX,NE,NYJ,CIN,WAS,ARI,TB,KC,NO
3,Spread,14,3.5,4,5,1.5,-5.5,12.5,-7.5,3.5,2,3,-12.5,2.5
4,abfarias,PHI,LAC,GB,MIA,LV,IND,BUF,CAR,MIN,SEA,LAR,TEN,BAL


In [19]:
week8_raw = pd.read_csv('week8_results.csv')
week8_raw.head()
#week8 = combine(week8_raw)
#week8.head()

,Player,BAL at TB,DEN at JAX,ARI at MIN,MIA at DET,CHI at DAL,LV at NO,CAR at ATL,NE at NYJ,PIT at PHI,TEN at HOU,NYG at SEA,SF at LAR,WAS at IND,GB at BUF,CIN at CLE
0,Winner,BAL,DEN,MIN,Mia,DAl,NO,ATL,NE,PHI,TEN,SEA,SF,WAS,BUG,CLE
1,Away team,BAL,DEN,ARI,MIA,CHI,LV,CAR,NE,PIT,TEN,NYG,SF,WAS,GB,CIN
2,Home Team,TB,JAX,MIN,DET,DAL,NO,ATL,NYJ,PHI,HOU,SEA,LAR,IND,BUF,CLE
3,Spread,1,-2.5,-3.5,3.5,-9.5,1.5,-4.5,2.5,-10.5,2,-3,1.5,-2.5,-11,3.5
4,abfarias,TB,JAX,MIN,MIA,CHI,NO,ATL,NYJ,PHI,TEN,NYG,SF,IND,BUF,CIN


In [10]:
for week in results:
    print(results[week].iloc[4])

Player        abfarias
Game 1      [BUF, 1.0]
Game 2     [ATL, 16.0]
Game 3     [NYJ, 15.0]
Game 4     [CIN, 14.0]
Game 5     [MIA, 13.0]
Game 6     [WAS, 12.0]
Game 7     [CHI, 11.0]
Game 8     [CAR, 10.0]
Game 9      [HOU, 9.0]
Game 10     [DET, 8.0]
Game 11     [LAC, 7.0]
Game 12     [MIN, 6.0]
Game 13     [TEN, 5.0]
Game 14     [ARI, 4.0]
Game 15     [DAL, 3.0]
Game 16     [SEA, 2.0]
Name: 4, dtype: object
Player           abfarias
LAC at KC      [LAC, 7.0]
CAR at NYG    [NYG, 10.0]
NE at PIT      [PIT, 5.0]
TB at NO        [TB, 6.0]
NYJ at CLE     [CLE, 9.0]
MIA at BAL     [BAL, 8.0]
WAS at DET     [WAS, 3.0]
IND at JAX    [IND, 11.0]
ATL at LAR    [LAR, 16.0]
SEA at SF      [SF, 12.0]
HOU at Den    [DEN, 14.0]
ARI at LV       [LV, 4.0]
CIN at DAL    [CIN, 13.0]
CHI at GB      [CHI, 2.0]
TEN at BUF    [BUF, 15.0]
MIN at PHI     [PHI, 1.0]
Name: 4, dtype: object
Player           abfarias
PIT at CLE     [PIT, 1.0]
LV at TEN     [TEN, 16.0]
CIN at NYJ    [CIN, 15.0]
BUF at MIA    [BU